In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
from modem import qpsk, chirp, wav

In [ ]:
rng = np.random.default_rng(seed=42)

def generate_pseudo_random_symbol():
    num_bytes = 1024
    data = bytes(rng.integers(255, size=num_bytes, dtype=np.uint8))

    qpsk_symbols = qpsk.qpsk_encode(data)
    ofdm_symbol = qpsk.encode_ofdm_symbol(qpsk_symbols)

    return ofdm_symbol

In [ ]:
white_noise_symbols = [generate_pseudo_random_symbol() for i in range(10)]
signal = np.concatenate((chirp.START_CHIRP, *white_noise_symbols, chirp.END_CHIRP))

In [ ]:
wav.generate_wav("chirp_white_noise.wav", signal)